**Combating Hate Speech Using NLP and Machine Learning**

**Objective**: Using NLP and ML, create a model to identify hate speech in Twitter.

**Problem Statement:** Twitter is one of the biggest platform where anybody and everybody can have their views heard. Some of these spread hate and negativity. Twitter is wary of its platform being used as a medium to spread hate.
You are a data scientist at Twitter, and you will help Twitter in identifying the tweets with hate speech and removing them from the platform. You will use NLP techniques, perform specific cleanup for tweets data, and make a robust model.

**Domain:** Social Media

**Analysis to be Done:** Clean up tweets and build a classification model by using NLP techniques, cleanup specific for tweets data, regularization and hyperparameter tuning using stratified K-fold and cross validation to get the best model.

**Content:**
id: identifier number of the tweet
Label: 0(non-hate)/1(hate)
Tweet: the text in the tweet

**Tasks:**
1. Load the tweets file using read_csv function from Pandas Package.
2. Get the tweets into a list for easy text cleanup and manipulation.
3. To cleanup:
          a. Normalize the casing.
          b. Using regular expressions, remove user handles. These begin with'@'
          c. Using regular expressions, remove URLs.
          d. Using TweetTokenizer from NLTK, tokenize the tweets into individual terms.
          e. Remove stop words.
          f. Remove redundant terms like 'amp','rt' etc.
          g. Remove '#' symbols from the tweet while retaining the term.
4. Extra cleanup by removing terms with a length of 1.
5. Check out the top terms in the tweets:
       a. First, get all the tokenized terms into one large list.
       b. Use the counter and find the 10 most common terms.
6. Data formatting for predictive modeling:
        a. Join the tokens back to form strings. This will be required for the vectorizers.
        b. Assign x and y.
        c. Perform train_test_split using sklearn.
7. We will TF-IDF values for the terms as a feature to get into a vector space model.
         a. Import TF-IDF vectorizer from sklearn.
         b. Instantiate with a maximum of 5000 terms in your vocabulary.
         c. Fit and apply on the train set.
         d. Apply on the test set.
8. Model building: Ordinary Logistic Regression
         a. Instantiate Logistic Regression from sklearn with default parameters.
         b. Fit into the train data.
         c. Make predictions for the train and the test set.
9. Model evaluation: Accuracy, recall, and f1 score.
         a. Report the accuracy on the train set.
         b. Report the recall on the train set: decent hight or low.
         c. Get the f1 score on the train set.
10. Train again with the adjustment and evaluate.
        a. Train the model on the train set.
        b. Evaluate the predictions on the train set: accuracy,recall and f1 score.
11. Regularization and Hyperparameter tuning:
          a. Import GridSearch and StratifiedKFold because of class imbalance.
          b. Provide the parameter grid to choose for 'C' and 'penalty' parameters.
          c. Use a balanced class weight while instantiating the logistic regression.
12. Find the parameters with the best recall in cross validation.
          a. Choose 'recall' as the metric for scoring.
          b. Choose stratified 4 fold cross validation scheme.
          c. Fit into the train set.
13. What are the best parameters?
14. Predict and evaluate using the best estimator.
          a. Use the best estimator from the grid search to make predictions on the test set.
          b. What is the recall on the on the test set for the toxic comments?
          c. What is the f1 score?


In [ ]:
import pandas as pd
import numpy as np
import os, re

 Read the CSV using Pandas

In [ ]:
inp_tweets0=pd.read_csv("Tweets_USA.csv")
inp_tweets0.head()


,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [ ]:
inp_tweets0.label.value_counts(normalize=True)

0    0.929854
1    0.070146
Name: label, dtype: float64

So we have around 7% of hate tweets in this data

In [ ]:
inp_tweets0.tweet.sample().values[0]

'on the train to london for the day   #familydays #dayout'

get the tweets into a list, for easy text clean up and manipulation

In [ ]:
tweets0 = inp_tweets0.tweet.values
#getting them into a list

In [ ]:
len(tweets0)

31962

In [ ]:
tweets0[:5]

array([' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run',
       "@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked",
       '  bihday your majesty',
       '#model   i love u take with u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦  ',
       ' factsguide: society now    #motivation'], dtype=object)

cleaning up the data
So as we can see here, The tweets contain:
URLs
Hashtags
user handles
'RT'

Normalising the casing

In [ ]:
tweets_lower = [txt.lower() for txt in tweets0]

In [ ]:
tweets_lower[:5]

[' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run',
 "@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked",
 '  bihday your majesty',
 '#model   i love u take with u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦  ',
 ' factsguide: society now    #motivation']

Remove uswer handels starting with '@'

In [ ]:
import re

In [ ]:
re.sub("@\w+","", "@Rahim this course rocks! http://rahimbaig.com/ai")

' this course rocks! http://rahimbaig.com/ai'

In [ ]:
tweets_nouser = [re.sub("@\w+","",twt) for twt in tweets_lower]

In [ ]:
tweets_nouser[:5]

['  when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run',
 "  thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked",
 '  bihday your majesty',
 '#model   i love u take with u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦  ',
 ' factsguide: society now    #motivation']

Now it looks a little bit cleaner as handels are removed

Remove URLs

In [ ]:
re.sub("\w+://\S+","", "@Rahim this course rocks! http://amazon.com/ai")

'@Rahim this course rocks! '

In [ ]:
tweets_nourl = [re.sub("\w+://\S+","", twt)

                for twt in tweets_nouser]

In [ ]:
tweets_nourl[:5]

['  when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run',
 "  thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked",
 '  bihday your majesty',
 '#model   i love u take with u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦  ',
 ' factsguide: society now    #motivation']

Tokenizing using Tweet from NLTK

In [ ]:
from nltk.tokenize import TweetTokenizer

In [ ]:
tkn = TweetTokenizer()

In [ ]:
print(tkn.tokenize(tweets_nourl[0]))

['when', 'a', 'father', 'is', 'dysfunctional', 'and', 'is', 'so', 'selfish', 'he', 'drags', 'his', 'kids', 'into', 'his', 'dysfunction', '.', '#run']


In [ ]:
tweet_token = [tkn.tokenize(sent) for sent in tweets_nourl]
print(tweet_token[31000])

['the', 'excitement', 'of', 'ordering', 'a', 'new', 'pole', 'outfit', '#poledance', '#poler', '#poledancer', '#poleoutfit', '#polefitness', '#healthy']


Remove stop words and punctuation

In [ ]:
# remove hastags also
from nltk.corpus import stopwords
from string import punctuation
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_nltk = stopwords.words("english")
stop_punct = list(punctuation)
stop_punct

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [ ]:
stop_punct.extend(['...','``',"''",".."])

In [ ]:
stop_context = ['rt','amp']

In [ ]:
stop_final = stop_nltk + stop_punct + stop_context

Function to

1.remove stop words from a single tokenized sentence

2.remove #s

3.remove terms with lenght=1

In [ ]:
def del_stop(sent):
    return [re.sub("#","",term) for term in sent if ((term not in stop_final) & (len(term)>1))]

In [ ]:
del_stop(tweet_token[4])


['factsguide', 'society', 'motivation']

In [ ]:
tweets_clean = [del_stop(tweet) for tweet in tweet_token]

Check the top terms in the tweets

In [ ]:
from collections import Counter

In [ ]:
term_list = []
for tweet in tweets_clean:
  term_list.extend(tweet)

In [ ]:
res = Counter(term_list)
res.most_common(10)

[('love', 2748),
 ('day', 2276),
 ('happy', 1684),
 ('time', 1131),
 ('life', 1118),
 ('like', 1047),
 ("i'm", 1018),
 ('today', 1013),
 ('new', 994),
 ('thankful', 946)]

Data formatting for predictive modelling

Join the tokens back into strings

In [ ]:
tweets_clean[30000]

['never', 'msg', 'first', 'dun', 'msg', 'first', 'disappointed']

In [ ]:
tweets_clean = ["".join(tweet) for tweet in tweets_clean]

In [ ]:
tweets_clean[30000]

'nevermsgfirstdunmsgfirstdisappointed'

Separate X and Y and perform train test split, 70-30

In [ ]:
len(tweets_clean)

31962

In [ ]:
len(inp_tweets0.label)

31962

In [ ]:
X = tweets_clean
y = inp_tweets0.label.values

In [ ]:
#train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)



Create a document term matrix using count vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(max_features = 5000)

In [ ]:
len(X_train), len(X_test)

(22373, 9589)

In [ ]:
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)



In [ ]:
X_train_bow.shape, X_test_bow.shape

((22373, 5000), (9589, 5000))

Model Building: Logistic Model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression()

In [ ]:
logreg.fit(X_train_bow, y_train)

LogisticRegression()

In [ ]:
y_train_pred = logreg.predict(X_train_bow)
y_test_pred = logreg.predict(X_test_bow)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
accuracy_score(y_train, y_train_pred)

0.9362177624815626

In [ ]:
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97     20815
           1       1.00      0.08      0.16      1558

    accuracy                           0.94     22373
   macro avg       0.97      0.54      0.56     22373
weighted avg       0.94      0.94      0.91     22373



Adjusting for class imbalance

In [ ]:
logreg = LogisticRegression(class_weight="balanced")

In [ ]:
logreg.fit (X_train_bow, y_train)

LogisticRegression(class_weight='balanced')

In [ ]:
y_train_pred = logreg.predict(X_train_bow)
y_test_pred = logreg.predict(X_test_bow)

In [ ]:
accuracy_score(y_train, y_train_pred)

0.9544540294104501

Accuracy score get improved after adjusting the imbalance.

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [ ]:
param_grid={
    'C':[0.01,0.1,1,10]
}

In [ ]:
classifier_lr = LogisticRegression(class_weight="balance")

In [ ]:
# Instantiate the grid search
grid_search=GridSearchCV(estimator=classifier_lr,
                        param_grid=param_grid,
                        cv=StratifiedKFold(4),
                        n_jobs=-1, verbose=1,scoring="recall")

In [ ]:
grid_search.fit(X_train_bow, y_train)

Fitting 4 folds for each of 4 candidates, totalling 16 fits


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
             estimator=LogisticRegression(class_weight='balance'), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10]}, scoring='recall', verbose=1)

In [ ]:
grid_search.best_estimator_

LogisticRegression(C=10, class_weight='balance')

using the best estimator to make predictions on test set

In [ ]:
y_test_pred=grid_search.best_estimator_.predict(X_test_bow)

In [ ]:
y_train_pred = grid_search.best_estimator_.predict(X_train_bow)

In [ ]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      8905
           1       0.99      0.14      0.25       684

    accuracy                           0.94      9589
   macro avg       0.96      0.57      0.61      9589
weighted avg       0.94      0.94      0.92      9589



When using classification models in machine learning, there are three common metrics that we use to assess the quality of the model:

1. Precision: Percentage of correct positive predictions relative to total positive predictions.

2. Recall: Percentage of correct positive predictions relative to total actual positives.

3. F1 Score: A weighted harmonic mean of precision and recall. The closer to 1, the better the model.

has context menu
